<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=1e813042a415fb4342e4818b65fdd52f0a5d100b10f01f35e210278db5aabf33
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-1000)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Mounted at /content/drive


In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

M    76
L    63
H    37
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0    93
1    83
Name: StarStock, dtype: int64

In [7]:
# v40/v40n stocks
mypf['Criteria'].value_counts()

V40N    39
V40     38
2T3Y     8
V200     6
SS       6
TBD      2
Name: Criteria, dtype: int64

In [8]:
# latest quarter
mypf['LatestQtr'].value_counts()

1    163
0     13
Name: LatestQtr, dtype: int64

In [9]:
# star + v40/v40n stocks
mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]['StarStock'].value_counts()

1    41
0    36
Name: StarStock, dtype: int64

In [10]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    141
1     35
Name: InPortfolio, dtype: int64

In [11]:
# star stocks in portfolio
mypf[(mypf['InPortfolio'] == 1)]['StarStock'].value_counts()

1    22
0    13
Name: StarStock, dtype: int64

In [12]:
# criteria of portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Criteria'].value_counts()

V40     11
V40N     8
2T3Y     8
SS       5
TBD      2
V200     1
Name: Criteria, dtype: int64

In [13]:
# strategies in portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Strategy'].value_counts()

ATH     20
2T3Y    12
Name: Strategy, dtype: int64

In [14]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [15]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [16]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    #stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0] 
    tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]  
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]       
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0] 
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [17]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-05 00:00:00+05:30,18069.0,^NSEI,2.11,0.56,0.12,18812.0,10806.0,9.28,4.0
2023-05-12 00:00:00+05:30,18315.0,^NSEI,3.18,0.59,0.31,18812.0,10806.0,6.21,3.0
2023-05-19 00:00:00+05:30,18203.0,^NSEI,2.32,0.54,0.48,18812.0,10806.0,7.61,3.0
2023-05-26 00:00:00+05:30,18499.0,^NSEI,3.79,0.41,0.42,18812.0,10806.0,3.91,2.0
2023-06-02 00:00:00+05:30,18529.0,^NSEI,3.80,0.34,0.84,18812.0,10806.0,3.53,2.0


In [18]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [19]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [20]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 35


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,925.0,-43.55,-4.15,-2.19,99.09,367.0,1354.78,45.2,18.50,M,59.0,1,0,2T3Y,2T3Y
VAIBHAVGBL.NS,300.0,-6.43,0.00,-0.33,95.83,222.0,320.64,35.4,47.30,M,-34.0,1,0,2T3Y,2T3Y
VALIANTORG.NS,611.0,7.03,-0.70,4.07,84.30,203.0,530.15,24.0,19.50,M,19.0,0,0,2T3Y,2T3Y
LUXIND.NS,1506.0,-3.65,-1.34,1.47,88.97,202.0,1499.87,21.4,33.00,M,-54.0,1,0,V40N,2T3Y
TEAMLEASE.NS,2230.0,-14.40,-2.19,-0.86,93.79,137.0,2239.86,NaN,33.60,M,NaN,1,1,V40N,2T3Y
AAVAS.NS,1395.0,-25.67,-2.45,-2.22,96.00,129.0,1405.00,NaN,25.70,M,NaN,1,1,2T3Y,2T3Y
INDIGOPNTS.NS,1467.0,11.93,-0.08,3.56,77.30,112.0,1408.25,104.9,52.90,M,50.0,1,1,V40N,2T3Y
JUBLFOOD.NS,493.0,-5.04,-0.77,1.32,85.24,83.0,468.21,90.5,92.30,H,-2.0,1,1,SS,ATH
IRCTC.NS,650.0,-1.74,0.00,0.66,56.67,79.0,629.08,77.4,52.60,H,32.0,1,1,SS,ATH


In [21]:
# top 10 from portfolio for SIP
tmp_df.sort_values(by = 'Upside%ATH', ascending=False).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,925.0,-43.55,-4.15,-2.19,99.09,367.0,1354.78,45.2,18.5,M,59.0,1,0,2T3Y,2T3Y
VAIBHAVGBL.NS,300.0,-6.43,0.00,-0.33,95.83,222.0,320.64,35.4,47.3,M,-34.0,1,0,2T3Y,2T3Y
VALIANTORG.NS,611.0,7.03,-0.70,4.07,84.30,203.0,530.15,24.0,19.5,M,19.0,0,0,2T3Y,2T3Y
LUXIND.NS,1506.0,-3.65,-1.34,1.47,88.97,202.0,1499.87,21.4,33.0,M,-54.0,1,0,V40N,2T3Y
TEAMLEASE.NS,2230.0,-14.40,-2.19,-0.86,93.79,137.0,2239.86,NaN,33.6,M,NaN,1,1,V40N,2T3Y
AAVAS.NS,1395.0,-25.67,-2.45,-2.22,96.00,129.0,1405.00,NaN,25.7,M,NaN,1,1,2T3Y,2T3Y
INDIGOPNTS.NS,1467.0,11.93,-0.08,3.56,77.30,112.0,1408.25,104.9,52.9,M,50.0,1,1,V40N,2T3Y


In [22]:
# top 5 from portfolio to book profit
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSINDBK.NS,1291.0,13.18,1.05,2.20,0.00,0.0,1288.70,22.8,13.4,H,41.0,1,1,SS,ATH
PIDILITIND.NS,2629.0,3.26,0.08,1.22,18.03,10.0,2442.09,94.6,105.0,H,-11.0,1,1,V40,ATH
ASIANPAINT.NS,3246.0,6.00,-0.36,1.25,19.09,10.0,3048.30,87.9,75.1,H,15.0,1,1,V40,ATH
BAJFINANCE.NS,7018.0,6.40,-0.20,1.99,18.06,13.0,6365.23,52.6,36.8,H,30.0,1,1,V40,ATH
RELIANCE.NS,2455.0,-0.61,-0.12,0.96,36.33,15.0,2464.49,29.1,24.9,H,14.0,1,1,V40,ATH
PNB.NS,52.0,11.49,2.13,0.00,23.53,15.0,50.60,NaN,17.2,L,NaN,1,1,SS,ATH
UJJIVANSFB.NS,36.0,34.07,3.70,3.45,20.69,17.0,33.60,16.8,1000.0,M,-5852.0,1,1,V40N,ATH


In [23]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [24]:
# star + v40/v40n stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
RAJESHEXPO.NS,563.0,-15.97,-0.30,-0.85,949.0,440.0,75.83,69.0,NaN,NaN,M,1,1,V40N,NaN
DIXON.NS,3930.0,7.78,-0.16,3.14,5750.0,1685.0,44.77,46.0,NaN,NaN,M,1,1,V40N,NaN
ANGELONE.NS,1312.0,-0.55,-0.08,2.03,1894.0,208.0,34.52,44.0,NaN,NaN,M,1,1,V40N,NaN
ASTRAZEN.NS,3526.0,7.82,0.37,0.46,4650.0,2454.0,51.18,32.0,NaN,NaN,M,1,1,V40N,NaN
BERGEPAINT.NS,649.0,7.31,0.00,0.99,858.0,540.0,65.72,32.0,1.0,0.0,M,1,1,V40,NaN
SUNTV.NS,454.0,-3.20,-0.21,0.70,577.0,388.0,65.08,27.0,NaN,NaN,M,1,1,V40N,NaN
CAPLIPOINT.NS,762.0,7.27,-0.42,1.79,949.0,400.0,34.06,25.0,NaN,NaN,M,1,1,V40N,NaN
OFSS.NS,3626.0,20.74,0.93,1.81,4392.0,2442.0,39.28,21.0,1.0,0.0,M,1,1,V40N,NaN


In [25]:
# top 5 from star + v40/v40n for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
RAJESHEXPO.NS,563.0,-15.97,-0.30,-0.85,949.0,440.0,75.83,69.0,NaN,NaN,M,1,1,V40N,NaN
SUNTV.NS,454.0,-3.20,-0.21,0.70,577.0,388.0,65.08,27.0,NaN,NaN,M,1,1,V40N,NaN
ANGELONE.NS,1312.0,-0.55,-0.08,2.03,1894.0,208.0,34.52,44.0,NaN,NaN,M,1,1,V40N,NaN
CAPLIPOINT.NS,762.0,7.27,-0.42,1.79,949.0,400.0,34.06,25.0,NaN,NaN,M,1,1,V40N,NaN
BERGEPAINT.NS,649.0,7.31,0.00,0.99,858.0,540.0,65.72,32.0,1.0,0.0,M,1,1,V40,NaN


In [26]:
# star + v40/v40n stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 10


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
MARICO.NS,552.0,9.07,0.20,1.19,569.0,325.0,6.97,3.0,1.0,0.0,M,1,1,V40,NaN
AKZOINDIA.NS,2469.0,11.58,1.27,0.67,2508.0,1653.0,4.56,2.0,NaN,NaN,M,1,1,V40,NaN
ITC.NS,443.0,24.52,1.97,1.71,450.0,146.0,2.30,2.0,1.0,1.0,H,1,1,V40,NaN
CERA.NS,7583.0,30.40,2.46,2.19,7690.0,2229.0,1.96,1.0,1.0,1.0,H,1,1,V40N,NaN
ICICIBANK.NS,940.0,5.67,0.67,1.32,954.0,333.0,2.25,1.0,1.0,1.0,H,1,1,V40,NaN
BAJAJ-AUTO.NS,4660.0,20.10,0.80,1.95,4660.0,2649.0,0.00,0.0,NaN,NaN,L,1,1,V40,NaN
EQUITASBNK.NS,88.0,48.31,3.39,2.70,88.0,33.0,0.00,0.0,1.0,1.0,M,1,1,V40N,NaN
HINDUNILVR.NS,2724.0,5.94,0.12,0.90,2728.0,1897.0,0.48,0.0,1.0,1.0,H,1,1,V40,NaN
POLYCAB.NS,3552.0,25.75,1.95,1.82,3552.0,793.0,0.00,0.0,1.0,0.0,M,1,1,V40N,NaN


In [27]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ADANITRANS.NS,798.0,-64.42,-5.48,-1.59,4105.0,235.0,85.45,414.0,1.0,-1.0,L,1,1,NaN,NaN
ADANIGREEN.NS,983.0,-37.85,-3.79,2.48,2970.0,462.0,79.23,202.0,-1.0,-1.0,L,1,1,NaN,NaN
ADANIENT.NS,2488.0,-13.72,-0.31,3.35,4165.0,270.0,43.06,67.0,1.0,0.0,M,1,1,NaN,NaN
EMAMILTD.NS,392.0,-7.78,-0.47,0.80,588.0,321.0,73.41,50.0,-1.0,1.0,L,1,1,V200,NaN
LTIM.NS,4998.0,8.69,0.26,0.82,7439.0,2334.0,47.82,49.0,1.0,0.0,M,1,1,V200,NaN
GODREJIND.NS,482.0,9.16,0.23,1.57,644.0,354.0,55.86,34.0,0.0,1.0,L,1,1,NaN,NaN
TATAPOWER.NS,216.0,0.70,-0.47,0.99,287.0,49.0,29.83,33.0,1.0,0.0,M,1,1,NaN,NaN
SBICARD.NS,914.0,11.74,-0.12,2.36,1133.0,673.0,47.61,24.0,1.0,0.0,M,1,1,NaN,NaN
CIPLA.NS,963.0,-5.99,-0.29,0.87,1173.0,701.0,44.49,22.0,1.0,0.0,M,1,1,NaN,NaN


In [28]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 13


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
BRITANNIA.NS,4654.0,12.36,1.11,0.88,4657.0,2945.0,0.18,0.0,1.0,1.0,H,1,1,NaN,NaN
EQUITASBNK.NS,88.0,48.31,3.39,2.70,88.0,33.0,0.00,0.0,1.0,1.0,M,1,1,V40N,NaN
HINDUNILVR.NS,2724.0,5.94,0.12,0.90,2728.0,1897.0,0.48,0.0,1.0,1.0,H,1,1,V40,NaN
POLYCAB.NS,3552.0,25.75,1.95,1.82,3552.0,793.0,0.00,0.0,1.0,0.0,M,1,1,V40N,NaN
TITAN.NS,2862.0,11.33,0.74,1.62,2862.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
CERA.NS,7583.0,30.40,2.46,2.19,7690.0,2229.0,1.96,1.0,1.0,1.0,H,1,1,V40N,NaN
ICICIBANK.NS,940.0,5.67,0.67,1.32,954.0,333.0,2.25,1.0,1.0,1.0,H,1,1,V40,NaN
AUBANK.NS,771.0,20.51,1.25,2.83,790.0,317.0,4.02,2.0,1.0,0.0,M,1,1,NaN,NaN
BHARTIARTL.NS,835.0,6.11,0.89,0.89,851.0,396.0,3.52,2.0,1.0,1.0,H,1,1,NaN,NaN


In [29]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ACC.NS,1810.0,-15.89,-1.02,0.29,2748.0,1259.0,63.00,52.0,0.0,0.0,L,0,0,NaN,NaN
PVRINOX.NS,1400.0,-14.01,-1.17,-0.74,1917.0,1348.0,90.86,37.0,1.0,-1.0,L,0,0,NaN,NaN
PGHH.NS,13650.0,-1.78,-0.28,0.01,15778.0,9190.0,32.30,16.0,0.0,0.0,L,0,0,V40,NaN
SIEMENS.NS,3567.0,16.27,1.30,0.78,3891.0,1181.0,11.96,9.0,1.0,1.0,H,0,0,NaN,NaN
NESTLEIND.NS,21970.0,11.73,0.61,1.68,22050.0,14623.0,1.08,0.0,1.0,1.0,H,0,0,V40,NaN


In [30]:
# stocks prospects 

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,


In [31]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,159.0,-9.8,-1.7,0.0,303.0,137.0,86.75,91.0,1.0,1.0,H,1,0,NaN,NaN


In [32]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,159.0,-9.8,-1.7,0.0,303.0,137.0,86.75,91.0,1.0,1.0,H,1,0,NaN,NaN


In [33]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 17


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
TITAN.NS,2862.0,11.33,0.74,1.62,2862.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
EQUITASBNK.NS,88.0,48.31,3.39,2.70,88.0,33.0,0.00,0.0,1.0,1.0,M,1,1,V40N,NaN
BRITANNIA.NS,4654.0,12.36,1.11,0.88,4657.0,2945.0,0.18,0.0,1.0,1.0,H,1,1,NaN,NaN
HINDUNILVR.NS,2724.0,5.94,0.12,0.90,2728.0,1897.0,0.48,0.0,1.0,1.0,H,1,1,V40,NaN
NESTLEIND.NS,21970.0,11.73,0.61,1.68,22050.0,14623.0,1.08,0.0,1.0,1.0,H,0,0,V40,NaN
CERA.NS,7583.0,30.40,2.46,2.19,7690.0,2229.0,1.96,1.0,1.0,1.0,H,1,1,V40N,NaN
ICICIBANK.NS,940.0,5.67,0.67,1.32,954.0,333.0,2.25,1.0,1.0,1.0,H,1,1,V40,NaN
ITC.NS,443.0,24.52,1.97,1.71,450.0,146.0,2.30,2.0,1.0,1.0,H,1,1,V40,NaN
BHARTIARTL.NS,835.0,6.11,0.89,0.89,851.0,396.0,3.52,2.0,1.0,1.0,H,1,1,NaN,NaN


In [34]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
HDFCLIFE.NS,581.0,6.44,0.37,2.23,757.0,461.0,59.46,30.0,1.0,1.0,H,1,0,V40,NaN
EICHERMOT.NS,3677.0,9.62,0.86,2.06,3850.0,2004.0,9.37,5.0,1.0,1.0,H,1,1,V40N,ATH
NESTLEIND.NS,21970.0,11.73,0.61,1.68,22050.0,14623.0,1.08,0.0,1.0,1.0,H,0,0,V40,NaN
TITAN.NS,2862.0,11.33,0.74,1.62,2862.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
ICICIBANK.NS,940.0,5.67,0.67,1.32,954.0,333.0,2.25,1.0,1.0,1.0,H,1,1,V40,NaN
M&M.NS,1331.0,4.86,0.24,1.22,1388.0,563.0,6.91,4.0,1.0,1.0,H,1,1,NaN,NaN
MARUTI.NS,9370.0,6.74,0.25,1.03,9528.0,6217.0,4.77,2.0,1.0,1.0,H,1,0,NaN,NaN
HAVELLS.NS,1337.0,8.79,0.08,0.98,1456.0,627.0,14.35,9.0,1.0,1.0,H,1,0,V40,NaN
HINDUNILVR.NS,2724.0,5.94,0.12,0.90,2728.0,1897.0,0.48,0.0,1.0,1.0,H,1,1,V40,NaN


In [35]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
TATACONSUM.NS,797.0,5.42,-0.13,1.35,866.0,454.0,16.75,9.0,1.0,1.0,H,1,1,SS,ATH
INDUSTOWER.NS,159.0,-9.80,-1.70,0.00,303.0,137.0,86.75,91.0,1.0,1.0,H,1,0,NaN,NaN


In [36]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 41


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ADANITRANS.NS,798.0,-64.42,-5.48,-1.59,4105.0,235.0,85.45,414.0,1.0,-1.0,L,1,1,NaN,NaN
PEL.NS,788.0,-10.87,-5.66,1.24,2866.0,635.0,93.14,264.0,-1.0,0.0,L,1,0,NaN,NaN
ADANIGREEN.NS,983.0,-37.85,-3.79,2.48,2970.0,462.0,79.23,202.0,-1.0,-1.0,L,1,1,NaN,NaN
IBREALEST.NS,68.0,-5.97,0.00,1.67,191.0,47.0,85.42,181.0,-1.0,-1.0,L,1,0,NaN,NaN
JCHAC.NS,1033.0,-14.75,-2.06,-0.82,2837.0,1017.0,99.12,175.0,NaN,NaN,M,1,0,V40N,NaN
LALPATHLAB.NS,2014.0,-8.70,-0.68,0.93,4186.0,1748.0,89.09,108.0,NaN,NaN,M,1,0,V40N,NaN
BIOCON.NS,239.0,-7.27,-1.16,1.30,483.0,199.0,85.92,102.0,1.0,0.0,M,1,0,NaN,NaN
SFL.NS,1014.0,-19.16,-1.67,-0.86,1972.0,623.0,71.02,94.0,NaN,NaN,M,1,0,V40N,NaN
ZEEL.NS,194.0,-16.57,-1.29,0.00,367.0,164.0,85.22,89.0,-1.0,1.0,L,1,0,NaN,NaN


In [37]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
PVRINOX.NS,1400.0,-14.01,-1.17,-0.74,1917.0,1348.0,90.86,37.0,1.0,-1.0,L,0,0,NaN,NaN


In [38]:
# save the output
df_prec_dev = df_prec_dev.reset_index()
df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [39]:
# nifty in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-05 00:00:00+05:30,787.0,BHARTIARTL.NS,1.72,0.78,0.26,851.0,396.0,14.07,8.0
2023-05-12 00:00:00+05:30,792.0,BHARTIARTL.NS,1.92,0.77,0.26,851.0,396.0,12.97,7.0
2023-05-19 00:00:00+05:30,806.0,BHARTIARTL.NS,3.30,0.77,0.52,851.0,396.0,9.89,6.0
2023-05-26 00:00:00+05:30,818.0,BHARTIARTL.NS,4.46,0.77,0.52,851.0,396.0,7.25,4.0
2023-06-02 00:00:00+05:30,835.0,BHARTIARTL.NS,6.09,0.89,0.89,851.0,396.0,3.52,2.0
